In [20]:
import requests
import json
from datetime import datetime

In [21]:
class GitHubRepository:
    def __init__(self, name, owner,  description, homepage, license, forks, watchers, date_of_collection):
        self.name = name
        self.owner = owner
        self.description = description
        self.homepage = homepage
        self.license = license  # License could be another class with its own attributes
        self.forks = forks
        self.watchers = watchers
        self.date_of_collection = date_of_collection

    @staticmethod
    def parse(dict):
      name = dict['name'] if 'name' in dict else None
      owner = dict['owner']['login'] if 'owner' in dict and 'login' in dict['owner'] else None
      description = dict['description'] if 'description' in dict else None
      homepage = dict['homepage'] if 'homepage' in dict else None
      forks = dict['forks'] if 'forks' in dict else None
      watchers = dict['watchers'] if 'watchers' in dict else None
      license = dict['license']['name'] if 'license' in dict and dict['license'] != None and 'name' in dict['license'] else None

      #get current time in UTC
      1971

      time = datetime.now()
      formatted_time = time.strftime("%Y-%m-%dT%H:%M:%SZ")

      return GitHubRepository(name, owner, description, homepage, license, forks, watchers, formatted_time)

    def __str__(self):
        return f"{self.owner}/{self.name}: license = [{self.license}] homepage = [{self.homepage}] forks = ({self.forks}) watchers = ({self.watchers}) date of collection: {self.date_of_collection}"

In [22]:
class PullRequest:
  def __init__(self,user, title, number, body, state, created_at, closed_at):
    self.user = user
    self.title = title
    self.number = number
    self.body = body
    self.state = state
    self.created_at = created_at
    self.closed_at = closed_at

  @staticmethod
  def parse(dict):
    title = dict['title'] if 'title' in dict else None
    user = dict['user']['login'] if 'user' in dict and 'login' in dict['user'] else None
    number = dict['number'] if 'number' in dict else None
    body = dict['body'] if 'body' in dict else None
    state = dict['state'] if 'state' in dict else None
    created_at = dict['created_at'] if 'created_at' in dict else None
    closed_at = dict['closed_at'] if 'closed_at' in dict else None

    return PullRequest(user,title,number,body,state,created_at,closed_at)

  def __str__(self):
    return f"{self.user}/{self.title}: ({self.state}) ({self.number}) ({self.created_at}) ({self.closed_at})"

In [23]:
class PRStatus:
  def __init__(self,number,commits,additions,deletions,changed_files):
    self.number = number
    self.commits = commits
    self.additions = additions
    self.deletions = deletions
    self.changed_files = changed_files

  @staticmethod
  def parse(dict):
    number = dict['number'] if 'number' in dict else None
    commits = dict['commits'] if 'commits' in dict else None
    additions = dict['additions'] if 'additions' in dict else None
    deletions = dict['deletions'] if 'deletions' in dict else None
    changed_files = dict['changed_files'] if 'changed_files' in dict else None

    return PRStatus(number,commits,additions,deletions,changed_files)

  def __str__(self):
    return f"PullRequest #{self.number}: commits = ({self.commits}) additions = ({self.additions}) deletions = ({self.deletions}) file changes = ({self.changed_files})"

In [24]:
class UserInfo:
  def __init__(self, name, no_of_repos, followers, following):
    self.name = name
    self.no_of_repos = no_of_repos
    self.followers = followers
    self.following = following

  @staticmethod
  def parse(dict):
    name = dict['name'] if 'name' in dict else None
    no_of_repos = dict['public_repos'] if 'public_repos' in dict else None
    followers = dict['followers'] if 'followers' in dict else None
    following = dict['following'] if 'following' in dict else None

    return UserInfo(name, no_of_repos, followers, following)

  def set_contributions(self,contributions):
    self.contributions = contributions



  def __str__(self):
    return f'{self.name}: repos = {self.no_of_repos}, followers = {self.followers}, following = {self.following}, contributions in last year = {self.contributions}'

In [25]:
# Get Repository

owner = "debsourav33"
repo_name = "Leetcode-Solutions"

def get_repository(owner, repo) -> GitHubRepository :

  url = f"https://api.github.com/repos/{owner}/{repo}"
  response = requests.get(url)

  if response.status_code != 200:
    return None

  resp_dict = response.json()
  pretty_response = json.dumps(resp_dict, indent = 4)
  #print(pretty_response)

  return GitHubRepository.parse(resp_dict)

github_repo = get_repository('ramnes', 'notion-sdk-py')
print(github_repo)

ramnes/notion-sdk-py: license = [MIT License] homepage = [https://ramnes.github.io/notion-sdk-py] forks = (114) watchers = (1373) date of collection: 2023-11-18T02:31:07Z


In [26]:
# Get Pull Request

owner = "jabref"
repo = "jabref"

def getPullRequests(owner, repo):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
  print(url)
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Status code: {response.status_code}")
      return None
  else:
    resp_list = response.json()
    #print(len(resp_list))
    ret = []
    for item in resp_list:
      pr= PullRequest.parse(item)
      ret.append(pr)
      #print(pr)
    return ret


def getPullRequestInfo(owner, repo, number):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{number}"
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Failed to retrieve pull requests. Status code: {response.status_code}")
      return None
  else:
    resp_dict = response.json()
    #print(type(resp_dict))
    #print(resp_dict)
    pr_info = PRStatus.parse(resp_dict)
    #print(pr_info)
    return pr_info



pull_requests = getPullRequests(owner,repo)
limit = 5 #so that we don't reach api call limit

for i in range(min(limit,len(pull_requests))):
  pr = pull_requests[i]
  if pr.number != None:
    pr_info = getPullRequestInfo(owner, repo, pr.number)
    print(pr_info)


https://api.github.com/repos/jabref/jabref/pulls
PullRequest #10646: commits = (24) additions = (118672) deletions = (49651) file changes = (1975)
PullRequest #10620: commits = (7) additions = (70) deletions = (9) file changes = (10)
PullRequest #10610: commits = (4) additions = (167) deletions = (91) file changes = (12)
PullRequest #10592: commits = (29) additions = (474) deletions = (13) file changes = (17)
PullRequest #10591: commits = (22) additions = (142) deletions = (30) file changes = (10)


In [27]:
# Get User Info

user = "jabref"


def getUserInfo(user):
  url = f'https://api.github.com/users/{user}'

  user_info = None
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Status code for {url}: {response.status_code}")
    return None
  else:
    resp_dict = response.json()
    user_info = UserInfo.parse(resp_dict)
    #print(json.dumps(resp_dict,indent = 4))

  #contributions
  url = f"https://api.github.com/users/{user}/events"
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Status code for {url}: {response.status_code}")
  else:
    events = response.json()
    contributions = 0

    for event in events:
      if event.get('type','null') == 'PushEvent':
        #print(json.dumps(event, indent=4))

        #check if the contribution was made this year
        created_at = event.get('created_at','')
        if created_at != '':
          created_year = datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%SZ').year
          curr_year = datetime.now().year

          if curr_year == created_year:
            contributions += 1


    if user_info != None:
      user_info.set_contributions(contributions)

    return user_info

user_info = getUserInfo(user)
print(user_info)

JabRef e.V.: repos = 45, followers = 63, following = 0, contributions in last year = 1
